In [1]:
#Import Packages
from selenium import webdriver
import time
import requests
import pandas as pd
import os
from bs4 import BeautifulSoup

In [2]:
#Import Packages

from selenium.webdriver.support.select import Select
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.common.by import By
from selenium.webdriver.support import expected_conditions as EC
from selenium.common.exceptions import NoSuchElementException

In [3]:
HEADERS = ({'User-Agent':'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/123.0.0.0 Safari/537.36', 'Accept-Language': 'en-US, en;q=0.5'})

In [4]:
url = "https://www.jobsforher.com/jobs?keyword=Software+Engineering%2CQA%2CQuality+Control%2CCustomer+Service+Tech+Support&job_skills=&company_keyword=&button_type=&min_exp=0&max_exp=1"

In [5]:
driver = webdriver.Chrome() # Initialize Chrome WebDriver
driver.implicitly_wait(2) # Wait for up to 2 seconds for elements to appear
driver.get(url) # Open the URL

In [6]:
def get_role_name(soup):

    try:
        Job_name = soup.find('h1', class_='mb-1 emp_wel_text22').text

    except AttributeError:
        Job_name = ""
    return Job_name

def get_company_name(soup):
    try:
        div_element = soup.find('div', class_='jobs-fixed-header')
        # Extract the company name
        Company_name = div_element.find('h2').text.strip()

    except AttributeError:
        Company_name = ""
    return Company_name

def get_location(soup):
    try:
        location_element = soup.find('div', class_='mb-1 jd-title3-18 d-none d-md-block')

        # Extract the text content after stripping unnecessary characters
        location_name = location_element.text.strip().split('&nbsp;&nbsp;')[-1]
        location = location_name.split()[0]

    except AttributeError:
        location = ""
    return location

def get_jd(soup):
    try:
        job_description = soup.find('div', class_='descriptionless').text.strip()

    except AttributeError:
        job_description = ""
    return job_description

def get_salary(soup):
    try:
        salary_element = soup.find('p', string='Salary')
        salary = salary_element.find_next_sibling('p').text.strip()

    except AttributeError:
        salary = ""
    return salary

def get_skills(soup):
    try:
        skills_element = soup.find('p', string='Key Skills Required')
        skills = skills_element.find_next_sibling('span').text.strip()

    except AttributeError:
        skills = ""
    return skills

In [7]:
d = {"Company_name":[], "Role":[],  "Skills":[], "Job_description":[],"Salary":[], "Location":[], "Application_link":[]}
print(d)

{'Company_name': [], 'Role': [], 'Skills': [], 'Job_description': [], 'Salary': [], 'Location': [], 'Application_link': []}


In [8]:
for i in range(3): # Adjust the range as required
    try:
        button_element = driver.find_element(By.ID, "my-job-load-more") # Find button element
        button_element.click() # Click the button
    except NoSuchElementException:
        print("Button no longer exists.")
        break
    time.sleep(4)

In [9]:
html_content = driver.page_source # Get the HTML content of the current page
soup = BeautifulSoup(html_content, 'html.parser') # Parse the HTML content using BeautifulSoup
links = soup.find_all("a", attrs={'class':'text-left mb-1'}) # Find all links with the specified class
links_list = []
for link in links:
    links_list.append(link.get('href'))

In [10]:
for link in links_list:
        new_webpage = requests.get("https://www.jobsforher.com/" + link, headers=HEADERS) # Send request to access the page
        new_link = "https://www.jobsforher.com/"+link # New link 

        new_soup = BeautifulSoup(new_webpage.content, "html.parser") # Parse the HTML content of the new webpage
        # Call functions to extract the details
        d["Company_name"].append(get_company_name(new_soup))
        d["Role"].append(get_role_name(new_soup))
        d["Location"].append(get_location(new_soup))
        d["Salary"].append(get_salary(new_soup))
        d["Skills"].append(get_skills(new_soup))
        d["Job_description"].append(get_jd(new_soup))
        d["Application_link"].append(new_link)
        



In [11]:
print(d)

{'Company_name': ['Forcepoint', 'Industrial Engineering Consortium', 'Promotup HR Solutions', 'Sycmes Services Pvt Ltd', 'Sycmes Services Pvt Ltd', 'Softpulse Infotech', 'Syniverse', 'sky career air services pvt ltd', 'Syniverse', 'SunPharma', 'Syniverse', 'Syniverse', 'Syniverse', 'Absolute Engineering Company', 'Accureq Managed Services', 'DAZN', 'DAZN', 'Corpseed', 'MERIDA TECH MINDS PVT LTD', 'Equinix', 'DAZN', 'Syniverse', 'Midbrains Technology', 'Midbrains Technology', 'Syniverse'], 'Role': ['QA Engineer (Manual)', 'Field Service Engineer', 'Customer Service Executive', 'Customer Service Executive', 'Customer Service Executive', 'Customer Support Representative', 'Customer Service Specialist I', 'CUSTOMER SERVICE ASSOCIATE  JOB IN AIRLINE', 'QA Engineer - ETL', 'QA Associate - Documentation', 'Sr QA Engineer', 'Sr QA Engineer', 'Sr QA Engineer', 'Export Executive', 'Customer Sales Executive', 'Mobile Engineering Manager - Android', 'Broadcast Support Engineer', 'Customer Care Age

In [12]:
df = pd.DataFrame.from_dict(d)

  # Save DataFrame to CSV file
csv_file_path = "job_data_jobsforher2.csv"
df.to_csv(csv_file_path, index=False)